# Convert NeRF to Timeloop Layer Shapes

In [1]:
import pytorch2timeloop
import os

import sys
sys.path.append("../") # go to parent dir

from accelerating_nerfs.models import VanillaNeRF

In [2]:
# Using vanilla NeRF which are MLPs
model = VanillaNeRF()

"VanillaNeRF Architecture", [model.modules]

('VanillaNeRF Architecture',
 [<bound method Module.modules of VanillaNeRF(
    (posi_encoder): SinusoidalEncoder()
    (view_encoder): SinusoidalEncoder()
    (mlp): NerfMLP(
      (base): MLP(
        (hidden_activation): ReLU()
        (output_activation): Identity()
        (hidden_layers): ModuleList(
          (0): Linear(in_features=63, out_features=256, bias=True)
          (1): Linear(in_features=256, out_features=256, bias=True)
          (2): Linear(in_features=256, out_features=256, bias=True)
          (3): Linear(in_features=256, out_features=256, bias=True)
          (4): Linear(in_features=256, out_features=256, bias=True)
          (5): Linear(in_features=319, out_features=256, bias=True)
          (6): Linear(in_features=256, out_features=256, bias=True)
          (7): Linear(in_features=256, out_features=256, bias=True)
        )
      )
      (sigma_layer): DenseLayer(
        (hidden_activation): ReLU()
        (output_activation): Identity()
        (hidden_layers

In [3]:
input_shape = (1, 3)
batch_size = 2 ** 14
save_dir = "../architectures/layer_shapes"
assert os.path.exists(save_dir)

print(f"input_shape = {input_shape}, batch_size = {batch_size}")

input_shape = (1, 3), batch_size = 16384


In [4]:
pytorch2timeloop.convert_model(
    model=model,
    input_size=input_shape,
    batch_size=batch_size,
    model_name="Nerf",
    save_dir=save_dir,
    convert_fc=True,
    exception_module_names=[],
)
print("Success, check results in workloads/nerf/*")
!ls workloads/nerf

unknown module type <class 'accelerating_nerfs.models.SinusoidalEncoder'>
unknown module type <class 'accelerating_nerfs.models.SinusoidalEncoder'>
unknown module type <class 'torch.nn.modules.linear.Identity'>
unknown module type <class 'accelerating_nerfs.models.MLP'>
unknown module type <class 'torch.nn.modules.linear.Identity'>
unknown module type <class 'accelerating_nerfs.models.DenseLayer'>
unknown module type <class 'torch.nn.modules.linear.Identity'>
unknown module type <class 'accelerating_nerfs.models.DenseLayer'>
unknown module type <class 'torch.nn.modules.linear.Identity'>
unknown module type <class 'accelerating_nerfs.models.MLP'>
unknown module type <class 'accelerating_nerfs.models.NerfMLP'>
unknown module type <class 'accelerating_nerfs.models.VanillaNeRF'>


Success, check results in workloads/nerf/*
nerf_layer1.yaml   nerf_layer12.yaml  nerf_layer4.yaml	nerf_layer7.yaml
nerf_layer10.yaml  nerf_layer2.yaml   nerf_layer5.yaml	nerf_layer8.yaml
nerf_layer11.yaml  nerf_layer3.yaml   nerf_layer6.yaml	nerf_layer9.yaml
